In [1]:
from lines_generator import dataGenerator
import numpy as np
import keras
from keras import Sequential
from keras.layers import Dense,Conv2D, MaxPooling2D, Flatten

Using TensorFlow backend.


In [2]:
patche_size = 17
input_dim = patche_size**2

## ANN Generator

In [3]:
gen_train = dataGenerator(50,'train',prob_peaks=0.2,prob_near_peaks=0.8)
gen_test = dataGenerator(10,'test',prob_peaks=0.2,prob_near_peaks=0.8)

In [4]:
# create model
# activations = ['relu','softmax','sigmoid']
model = Sequential()
model.add(Dense(200, input_dim=input_dim, activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

In [5]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
model.fit_generator(gen_train, samples_per_epoch=64, nb_epoch=50)

/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=64, epochs=50)`
  """Entry point for launching an IPython kernel.


Epoch 1/50
64/64 [==============================] - 11s 174ms/step - loss: 0.4067 - acc: 0.8027
Epoch 2/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2906 - acc: 0.8613
Epoch 3/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2858 - acc: 0.8408
Epoch 4/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2779 - acc: 0.8506
Epoch 5/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2938 - acc: 0.8516
Epoch 6/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2514 - acc: 0.8789
Epoch 7/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2470 - acc: 0.8770
Epoch 8/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2548 - acc: 0.8779
Epoch 9/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2594 - acc: 0.8760
Epoch 10/50
64/64 [==============================] - 0s 3ms/step - loss: 0.2545 - acc: 0.8623
Epoch 11/50
64/64 [==============================] - 0s 3ms/step -

In [7]:
score = model.evaluate_generator(gen_test,steps=128)
score

[0.19393028672493529, 0.90966796875]

In [8]:
pred_test = model.predict_generator(gen_test,steps=128,verbose=1)

128/128 [==============================] - 0s 2ms/step


In [9]:
pred_test

array([[0.00312695, 0.9968731 ],
       [0.90895385, 0.09104614],
       [0.00515875, 0.9948413 ],
       ...,
       [0.03850157, 0.9614984 ],
       [0.79164743, 0.20835258],
       [0.00457611, 0.99542385]], dtype=float32)

## Save

In [10]:
model.save('models/model3.h5')